## Install packages

In [ ]:
# @title
# Manage data and statistics
import numpy as np
from numpy.random import default_rng, SeedSequence
import pandas as pd

from scipy import stats
from scipy.stats import norm, skewnorm, yeojohnson, boxcox, zscore
from scipy.stats.mstats import winsorize

# Plot data
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import seaborn as sns
sns.set(style="white")

# Scale variables
from sklearn.preprocessing import scale, StandardScaler
from sklearn.preprocessing import minmax_scale, MinMaxScaler
from sklearn.preprocessing import maxabs_scale, MaxAbsScaler
from sklearn.preprocessing import robust_scale, RobustScaler

# Transform variables
from sklearn.preprocessing import quantile_transform, QuantileTransformer
from sklearn.preprocessing import power_transform, PowerTransformer

# Encode categorical variables
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder

# Discretize continuous variables
from sklearn.preprocessing import KBinsDiscretizer

# Impute missing values
from sklearn.impute import SimpleImputer, KNNImputer, MissingIndicator
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

## Read data

In [ ]:
data = {
    'id': [1000, 1001, 1002, 1003, 1004, 1005, 1006],
    'date': pd.date_range('4/25/2020', periods=7, freq='D'),
    'age': [21, 56, 33, 48, 27, 42, 32],
    'income': [67000, 220000, 97000, 166000, 81000, 157000, 96000],
    'gender': ['Male', 'Female', 'Female', 'Male', 'Male', 'Female', 'Female'],
    'education': ['Bachelors', 'PhD', 'Masters', 'Masters', 'Bachelors', 'Bachelors', 'Bachelors'],
    'passed': [False, True, True, True, False, False, True],
    'measurement': np.random.randn(7).round(2)
}

df = pd.DataFrame(data)
df

,id,date,age,income,gender,education,passed,measurement
0,1000,2020-04-25,21,67000,Male,Bachelors,False,-0.23
1,1001,2020-04-26,56,220000,Female,PhD,True,0.02
2,1002,2020-04-27,33,97000,Female,Masters,True,0.95
3,1003,2020-04-28,48,166000,Male,Masters,True,0.60
4,1004,2020-04-29,27,81000,Male,Bachelors,False,-0.49
5,1005,2020-04-30,42,157000,Female,Bachelors,False,0.60
6,1006,2020-05-01,32,96000,Female,Bachelors,True,-0.02


## Insert missing values into data

In [ ]:
# Insert missing values into the dataset
df2 = df.copy()
df2.iloc[[3, 5], [2, 4, 6, 7]] = np.nan
df2

,id,date,age,income,gender,education,passed,measurement
0,1000,2020-04-25,21.0,67000,Male,Bachelors,False,-0.23
1,1001,2020-04-26,56.0,220000,Female,PhD,True,0.02
2,1002,2020-04-27,33.0,97000,Female,Masters,True,0.95
3,1003,2020-04-28,NaN,166000,NaN,Masters,NaN,NaN
4,1004,2020-04-29,27.0,81000,Male,Bachelors,False,-0.49
5,1005,2020-04-30,NaN,157000,NaN,Bachelors,NaN,NaN
6,1006,2020-05-01,32.0,96000,Female,Bachelors,True,-0.02


## 1. Deal with missing values

## 1a. Create indicator variable for missing values in `age` column

In [ ]:
df2['age_imputed'] = np.where(df2['age'].isna(), 1.0, 0.0)
df2

,id,date,age,income,gender,education,passed,measurement,age_imputed
0,1000,2020-04-25,21.0,67000,Male,Bachelors,False,-0.23,0.0
1,1001,2020-04-26,56.0,220000,Female,PhD,True,0.02,0.0
2,1002,2020-04-27,33.0,97000,Female,Masters,True,0.95,0.0
3,1003,2020-04-28,NaN,166000,NaN,Masters,NaN,NaN,1.0
4,1004,2020-04-29,27.0,81000,Male,Bachelors,False,-0.49,0.0
5,1005,2020-04-30,NaN,157000,NaN,Bachelors,NaN,NaN,1.0
6,1006,2020-05-01,32.0,96000,Female,Bachelors,True,-0.02,0.0


## 1b. Fill in missing values in `age` column using [`pandas`](https://www.statology.org/pandas-fillna-with-mean/)

In [ ]:
df2['age_imp_w_pandas'] = df2['age']

df2.loc[df2['age_imp_w_pandas'].isnull(), 'age_imp_w_pandas'] = df2['age_imp_w_pandas'].mean()

df2

,id,date,age,income,gender,education,passed,measurement,age_imputed,age_imp_w_pandas
0,1000,2020-04-25,21.0,67000,Male,Bachelors,False,-0.23,0.0,21.0
1,1001,2020-04-26,56.0,220000,Female,PhD,True,0.02,0.0,56.0
2,1002,2020-04-27,33.0,97000,Female,Masters,True,0.95,0.0,33.0
3,1003,2020-04-28,NaN,166000,NaN,Masters,NaN,NaN,1.0,33.8
4,1004,2020-04-29,27.0,81000,Male,Bachelors,False,-0.49,0.0,27.0
5,1005,2020-04-30,NaN,157000,NaN,Bachelors,NaN,NaN,1.0,33.8
6,1006,2020-05-01,32.0,96000,Female,Bachelors,True,-0.02,0.0,32.0


## 1c. Fill in missing values in `age` column using [`scikit-learn`](https://scikit-learn.org/stable/modules/impute.html)

In [ ]:
# Fill in missing values with column mean
df2['age_imp_w_sklearn'] = df2['age']

imp = SimpleImputer(missing_values=np.nan, strategy='mean')

df2['age_imp_w_sklearn'] = pd.Series( imp.fit_transform(df2['age'].values.reshape(-1, 1)).flatten() )

df2

,id,date,age,income,gender,education,passed,measurement,age_imputed,age_imp_w_pandas,age_imp_w_sklearn
0,1000,2020-04-25,21.0,67000,Male,Bachelors,False,-0.23,0.0,21.0,21.0
1,1001,2020-04-26,56.0,220000,Female,PhD,True,0.02,0.0,56.0,56.0
2,1002,2020-04-27,33.0,97000,Female,Masters,True,0.95,0.0,33.0,33.0
3,1003,2020-04-28,NaN,166000,NaN,Masters,NaN,NaN,1.0,33.8,33.8
4,1004,2020-04-29,27.0,81000,Male,Bachelors,False,-0.49,0.0,27.0,27.0
5,1005,2020-04-30,NaN,157000,NaN,Bachelors,NaN,NaN,1.0,33.8,33.8
6,1006,2020-05-01,32.0,96000,Female,Bachelors,True,-0.02,0.0,32.0,32.0


## 2. Handle categorical data

## 2a. Create dummy variables based on the `education` column using [`pandas`](https://www.statology.org/pandas-get-dummies/)

In [ ]:
# To keep the new columns, set the result equal to a new variable, e.g. df3
pd.get_dummies(data=df2,
               columns=['education'],
               prefix=['edu'],
               dtype=float)

,id,date,age,income,gender,passed,measurement,age_imputed,age_imp_w_pandas,age_imp_w_sklearn,edu_Bachelors,edu_Masters,edu_PhD
0,1000,2020-04-25,21.0,67000,Male,False,-0.23,0.0,21.0,21.0,1.0,0.0,0.0
1,1001,2020-04-26,56.0,220000,Female,True,0.02,0.0,56.0,56.0,0.0,0.0,1.0
2,1002,2020-04-27,33.0,97000,Female,True,0.95,0.0,33.0,33.0,0.0,1.0,0.0
3,1003,2020-04-28,NaN,166000,NaN,NaN,NaN,1.0,33.8,33.8,0.0,1.0,0.0
4,1004,2020-04-29,27.0,81000,Male,False,-0.49,0.0,27.0,27.0,1.0,0.0,0.0
5,1005,2020-04-30,NaN,157000,NaN,NaN,NaN,1.0,33.8,33.8,1.0,0.0,0.0
6,1006,2020-05-01,32.0,96000,Female,True,-0.02,0.0,32.0,32.0,1.0,0.0,0.0


## 2b. Create dummy variables based on the `education` column using [`scikit-learn`](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-categorical-features)

In [ ]:
ohe = OneHotEncoder()
ohe.fit_transform( df['education'].values.reshape(-1, 1) ).toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

## 3. Bring features on the same scale

## 3a. Center and scale (i.e. standardize) the `income` column using [`pandas`](https://www.geeksforgeeks.org/how-to-standardize-data-in-a-pandas-dataframe/)

In [ ]:
print(f"Mean of income: {df2['income'].mean().round(1)}\nStd of income:   {df2['income'].std(ddof=0).round(1)}")

Mean of income: 126285.7
Std of income:   51607.6


In [ ]:
df2['income_cs'] = df2['income'].apply(lambda val: (val - df2['income'].mean()) / df2['income'].std(ddof=0))
df2

,id,date,age,income,gender,education,passed,measurement,age_imputed,age_imp_w_pandas,age_imp_w_sklearn,income_cs
0,1000,2020-04-25,21.0,67000,Male,Bachelors,False,-0.23,0.0,21.0,21.0,-1.148778
1,1001,2020-04-26,56.0,220000,Female,PhD,True,0.02,0.0,56.0,56.0,1.815900
2,1002,2020-04-27,33.0,97000,Female,Masters,True,0.95,0.0,33.0,33.0,-0.567469
3,1003,2020-04-28,NaN,166000,NaN,Masters,NaN,NaN,1.0,33.8,33.8,0.769543
4,1004,2020-04-29,27.0,81000,Male,Bachelors,False,-0.49,0.0,27.0,27.0,-0.877500
5,1005,2020-04-30,NaN,157000,NaN,Bachelors,NaN,NaN,1.0,33.8,33.8,0.595150
6,1006,2020-05-01,32.0,96000,Female,Bachelors,True,-0.02,0.0,32.0,32.0,-0.586846


## 3b. Center and scale (i.e. standardize) the `income` column using [`scikit-learn`](https://scikit-learn.org/stable/modules/preprocessing.html)

In [ ]:
scaler = StandardScaler().fit(df2['income'].values.reshape(-1, 1))

print(f"Mean of income: {np.round(scaler.mean_[0],1)}\nStd of income:   {np.round(scaler.scale_[0],1)}")

Mean of income: 126285.7
Std of income:   51607.6


In [ ]:
StandardScaler().fit_transform(df2['income'].values.reshape(-1, 1))

array([[-1.1487782 ],
       [ 1.8159    ],
       [-0.56746875],
       [ 0.76954299],
       [-0.87750046],
       [ 0.59515015],
       [-0.58684573]])